In [1]:
# Import Pandas
import pandas as pd

In [3]:
# Load the datasets
fake = pd.read_csv("data/Fake.csv")
true = pd.read_csv("data/True.csv")

In [4]:
# Check row counts
print("Fake dataset rows:", len(fake))
print("True dataset rows:", len(true))

Fake dataset rows: 23481
True dataset rows: 21417


In [5]:
# Check column names
print("Fake columns:", fake.columns.tolist())
print("True columns:", true.columns.tolist())

Fake columns: ['title', 'text', 'subject', 'date']
True columns: ['title', 'text', 'subject', 'date']


In [6]:
# Add label column
fake['label'] = 0  # Fake = 0
true['label'] = 1

In [7]:
# Merge the datasets
df = pd.concat([fake, true], ignore_index=True)

# Save the merged dataset
df.to_csv("data/news_dataset.csv", index=False)

In [8]:
# Verify the merge
print("Total rows after merge:", len(df))
print("Label distribution:\n", df['label'].value_counts())

Total rows after merge: 44898
Label distribution:
 label
0    23481
1    21417
Name: count, dtype: int64


In [9]:
# Check for missing values
print("Missing values:\n", df.isnull().sum())

# Look at a sample of 5 rows
print("\nSample of merged dataset:\n", df[['title', 'text', 'subject', 'label']].head(5))

Missing values:
 title      0
text       0
subject    0
date       0
label      0
dtype: int64

Sample of merged dataset:
                                                title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  label  
0  Donald Trump just couldn t wish all Americans ...    News      0  
1  House Intelligence Committee Chairman Devin Nu...    News      0  
2  On Friday, it was revealed that former Milwauk...    News      0  
3  On Christmas day, Donald Trump announced that ...    News      0  
4  Pope Francis used his annual Christmas Day mes...    News      0  
